# Verb Collocations with Atelic Time Duration Adverbials
## A Collostructional Approach

In this notebook, I develop procedures to select and analyze atelic, durative time phrases within Biblical Hebrew. In English these phrases are communicated with the "for" construction: e.g. X happened for three days. These phrases are important since they are used in telicity tests for lexical aspect. The fundamental assumption is that atelic durative time phrases do not normally combine with telic verb lexemes. In Fuhs' 2010 study, ["The Aspectual Coercion of the English Durative Adverbial,"](https://philpapers.org/rec/FUHTAC), he tests this assumption using collostruction analysis. This method, developed by Stefanowitsch and Gries ([see description here](http://www.linguistics.ucsb.edu/faculty/stgries/research/2003_AS-STG_Collostructions_IJCL.pdf)), tests the collocational preferences between a given construction and other lexical items. Those preferences are measured through a statistical test for significance, the Fisher's Exact test. Fuhs finds that 75% of the verb lexemes found to be statistically associated with atelic durative time adverbials do indeed have atelic aspect. Yet 25% of the verbs were truly telic. Fuhs also checks whether these 25% occur in the progressive aspect, which can construe telic action duratively. He finds that they do not, meaning that these cases are truly telic verb lexemes being construed as atelic. These findings suggest that the relationship between temporal adverbials and lexical aspect is more complex than it is sometimes described to be. 

In this notebook, we seek to apply Fuhs' methodology to the atelic durative time adverbials of Biblical Hebrew. The first step is to identify which forms in Biblical Hebrew indicate atelic, durative functions. The ETCBC Hebrew syntax database indicates adjunct phrases with a function of communicating "Time." For this pilot study, these phrases serve as the target of investigation.

In [the exploratory analysis](SBH_time_expressions.ipynb), I found that noun-phrase based time adverbials have two primary forms: NPs governed by a preposition (PP), and bare NPs. Amongst these two groups are further divisions between noun phrases modified with a quantifier or a demonstrative (including the demonstrative ה). A preliminary search leading up to this pilot study futher found that the durative atelic time construction in Biblical Hebrew appears to be conveyed by bare NP's without any demonstrative element, often with quantification or plurality.  

that durative, atelic time constructions in Biblical Hebrew are zero-marked, meaning it is not marked with a preposition or other modifiers, as is common in world languages:

> The most striking tendency observed in the expression of atelic-extent adverbials is the cross-linguistic tendency for zero expression or expression by means of a "minimal case". (Haspelmath 1997: 120)

As Haspelmath notes, English is among a few other Indo-European languages that employ a preposition such as "for" to indicate atelic duration.

## Forms in Biblical Hebrew

A few search constraints when querying  phrases marked with a "Time" function in the ETCBC can identify many atelic durative time adverbials. Those are: 1. exclude cases of preposition governance, 2. exclude cases where the definite article functions demonstratively, and 3. exclude cases with demonstratives. These constraints are surprisingly successful at identifying candidates for atelic time duration. 

In [6]:
# ETCBC's BHSA data
from tf.app import use

# stats & data-containers
import collections, random
import pandas as pd
import numpy as np
import scipy.stats as stats

# data visualizations
import seaborn as sns
sns.set(font_scale=1.5, style='whitegrid')
import matplotlib.pyplot as plt
from IPython.display import display

# load BHSA and heads data
A = use('bhsa', mod='etcbc/heads/tf', hoist=globals())
A.displaySetup(condenseType='clause') # configure Hebrew display

Using bhsa commit 7f353d587f4befb6efe1742831e28f301d2b3cea
  in /Users/cody/text-fabric-data/__apps__/bhsa
Using etcbc/bhsa/tf - c rv1.6 in /Users/cody/text-fabric-data
Using etcbc/phono/tf - c r1.2 in /Users/cody/text-fabric-data
Using etcbc/parallels/tf - c r1.2 in /Users/cody/text-fabric-data
Using etcbc/heads/tf - c rv.1.11 in /Users/cody/text-fabric-data


**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="provenance of BHSA = Biblia Hebraica Stuttgartensia Amstelodamensis">BHSA</a> <a target="_blank" href="https://annotation.github.io/text-fabric/Writing/Hebrew" title="('Hebrew characters and transcriptions',)">Character table</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/c/0_home.html" title="BHSA feature documentation">Feature docs</a> <a target="_blank" href="https://github.com/annotation/app-bhsa" title="bhsa API documentation">bhsa API</a> <a target="_blank" href="https://annotation.github.io/text-fabric/Api/Fabric/" title="text-fabric-api">Text-Fabric API 7.3.9</a> <a target="_blank" href="https://annotation.github.io/text-fabric/Use/Search/" title="Search Templates Introduction and Reference">Search Reference</a>

## Identifying "Durative" Time Phrases

### 1. Quantified Time Phrases

There seems to be a high selectivity for the durative function in time phrases that lack:

* prepositional governance [but what about עד-headed phrases or phrases with מן + עד relations?]
* determination or demonstrative

In [7]:
durative = A.search('''

phrase function=Time typ=NP
/without/
    word pdp=art|dem|prep
/-/

''')

  0.86s 496 results


In [8]:
A.show(durative, withNodes=True)



**result** *1*





**result** *2*





**result** *3*





**result** *4*





**result** *5*



...



**result** *97*





**result** *98*





**result** *99*





**result** *100*



**396 more results skipped** because we show a maximum of 100 results at a time